<a href="https://colab.research.google.com/github/AbeHandler/AbeHandler.github.io/blob/master/EricSaeedDataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json
import enum
import math

with open('data.jsonl', 'r') as f:
  lines = f.readlines()
  lines = [o.strip("\n").replace('"', "'").replace("'", '"').strip('"') for o in lines]

data = [json.loads(line) for line in lines]
df = pd.DataFrame(data)
df = df[df["number"].apply(lambda x: len(x) > 0)]

participants = df.groupby("guidString").count().reset_index()[["guidString", "turn"]]

ids = participants[participants["turn"] > 175]["guidString"].to_list()
len(ids)

23

In [13]:
import pandas as pd
import json
import enum
import math

id2clevelandanalysis = {}

with open('data.jsonl', 'r') as f:
    lines = f.readlines()
    lines = [o.strip("\n") for o in lines]
    data = [json.loads(line) for line in lines]

df = pd.DataFrame(data)
df = df[df["number"].apply(lambda x: len(x) > 0)]

# filter to people who did full turns here
s = len(df)
df = df[df["guidString"].apply(lambda x: x in ids)]
e = len(df)
assert s > e

df["CamZ"] = df["CamZ"].apply(lambda x: round(float(x)))
df["CamZ"] = df["CamZ"].apply(lambda x: -5 if x <= -5 else x) # if more neg then -5, just clip to -5
df["CamZ"] = df["CamZ"].apply(lambda x: 5 if x >= 5 else x) # if more pos then +5, just clip to +5
df["CamZ"] = df["CamZ"].apply(lambda x: 0 if abs(int(x)) <= 1 else x) # if w/in 1 of 0, clip to 0
df["ClevelandMode"] = df["ClevelandMode"] == "True"

assert len(df["CamZ"].apply(lambda x: math.floor(float(x))).unique()) == 3

class Position(enum.Enum):
    RIGHT = 5
    CENTER = 0
    LEFT = -5

df["position"] = df["CamZ"].apply(lambda x: Position(round(float(x))).name)
df["number"] = df["number"].apply(lambda x: float(x))
df["Lvar"] = df["Lvar"].apply(lambda x: float(x))
df["Rvar"] = df["Rvar"].apply(lambda x: float(x))

df["larger"] = df[["Lvar", "Rvar"]].max(axis=1)
df["smaller"] = df[["Lvar", "Rvar"]].min(axis=1)
df["data_pct"] = (df["smaller"] / df["larger"]) * 100

df = df.drop("larger", axis=1)
df = df.drop("smaller", axis=1)

# Miranda's question analyis
dfnC = df[~df["ClevelandMode"]].copy()
dfnC["number"] = dfnC["number"].apply(lambda x: int(x))
dfnC['Lvar'] = dfnC['Lvar'].apply(lambda x: x * 100)
dfnC["ncbias"] = dfnC["number"] - dfnC['Lvar']
dfnC["error"] = dfnC["number"] - dfnC['Lvar']
dfnC["error"] = dfnC["error"].abs()
dfnC = dfnC.groupby('position')['ncbias'].mean()
dfnC

position
CENTER    0.765015
LEFT      0.866025
RIGHT     1.916184
Name: ncbias, dtype: float64

In [14]:
dfC = df[df["ClevelandMode"]].copy()
dfC["bias"] = dfC["number"] - dfC["data_pct"]
dfC['right_bigger'] = (dfC['Rvar'] > dfC['Lvar'])
dfC['sideSmall'] = (dfC['right_bigger'] & (dfC['position'] == 'LEFT')) | (~dfC['right_bigger'] & (dfC['position'] == 'RIGHT'))
dfC = dfC[dfC["position"] != "CENTER"]
dfC.groupby(['sideSmall', "guidString"]).agg({'bias': ['mean', 'std']}).reset_index().sort_values("guidString")

dfC.groupby(['sideSmall']).agg({'bias': ['mean', 'std']}).reset_index()

sideSmall       bias           
                  mean        std
0     False -11.829119  21.231508
1      True  -0.884736  28.117766

In [15]:
from scipy.stats import ttest_ind
group_true = dfC[dfC['sideSmall'] == True]['bias']
group_false = dfC[dfC['sideSmall'] == False]['bias']

# Performing an independent t-test
t_stat, p_value = ttest_ind(group_true, group_false)

t_stat, p_value

(8.1318615790477, 9.3975297330309e-16)